### 본 프로젝트는 보유 종목에 대한 실시간 공시 및 뉴스를 가져올 수 있게한 프로그램입니다.
### 파일은 '국내펀드보유내역.xlsx'와 같은 양식을 지닌 파일이 있으면 작동이 가능합니다.
### 회사에서 사용한 프로그램이므로 회사 엑셀파일의 양식에 맞추느라 직접 사용하실 경우 약간의 수정이 필요합니다.(해당 양식은 capital IQ 양식입니다.)
### 실제 사용하실 경우, get_corp_list(self) 함수에서 data1,data2,data3 를 수정하셔야합니다.
### 회사의 니즈를 맞춰 양식 및 필요한 공시를 가져오기 위해 조건이 다소 일반적이지 않음은 양해부탁드립니다.
### 전체적인 알고리즘은 다음과 같습니다.

1. class 실행
2. 보유종목 엑셀파일을 열어 종목 저장 및 종목에 따른 업종 딕셔너리 생성
3. 보유종목에 따라 하나씩 dart에서 공시를 가져옵니다.
4. 이후 공시를 전부 가져오면, 네이버 실시간 뉴스를 가져옵니다.(중복뉴스 및 유사 뉴스는 임베딩을 통해 유사도를 측정 후 제거해주었습니다.)
5. 3-4과정을 프로그램이 꺼지지 않는 한 계속 반복합니다. 과부하를 막기위해 sleep을 걸어줘 240초 마다 서칭을 하게끔 해주었습니다.

In [ ]:
from tkinter import *
import pandas as pd
import numpy as np
from dart_fss.corp import corp_list
from selenium import webdriver

from sentence_transformers import SentenceTransformer,util
import pandas as pd
import numpy as np
import dart_fss as dart
import time
#from plyer import notification




news_dict={}
report_code = []

chrome_path = "C://Users\master//PycharmProjects//chromedriver.exe"
api_key = '' #api 키는 개인이 직접 받아와야합니다.

class searching_reports(): #클래스를 시작하면 먼저 공시를 가져오기 시작합니다. 이후 공시를 다 가지고 오면, 보유종목들의 실시간 뉴스를 가져오게끔 하였습니다.
    def __init__(self):

        self.last_dict={}
        global news_dict
        global chrome_path
        self.news_dict=news_dict

        self.get_corp_list() # 종목에 따른 업종 가져오는 함수
        #searching_reports().get_corp_list()

        report_code_old = {}

        while True:

            self.get_reports_list()

            for i in range(len(report_name)): #루프 반복
                if list(report_code.keys())[i] not in report_code_old.keys():

                    k="%s :: %s 공시가 올라왔습니다." % (report_code.get(list(report_code.keys())[i]), report_name.get(list(report_code.keys())[i]))
                    print(k)

            report_code_old = report_code


            ############################### 뉴스 가져오기 시작하는 코드
            for roop in range(5):
                self.last_dict = self.news_dict
                self.news_dict = news( corp=corp_list, news_dict=self.news_dict,chrome_path)
                # print("전종목을 검색 완료하였습니다.")
                if self.last_dict.values() != news_dict.values():
                    for i in self.news_dict.keys():

                        if i not in self.last_dict.keys():
                            print(news_dict[i])
                        else:
                            if self.last_dict[i] != self.news_dict[i]:
                                print(self.news_dict[i])

            time.sleep(240)

                # print("last_dict 갱신화 완료")
                # time.sleep()


    def get_corp_list(self): #업종에 대한 딕셔너리 생성 -> 공시에서 특정 업종의 공시는 불필요하게 자주 나오는 공시는 제거하기 위해 업종을 딕셔너리에 담아줬습니다. 모든 공시를 가져오려면 불필요합니다.

        global corp_list

        data_1 = pd.read_excel('국내펀드보유종목내역.xlsx', sheet_name='2010', usecols=['종목명', '업종'])
        data_2 = pd.read_excel('국내펀드보유종목내역.xlsx', sheet_name='5220', usecols=['종목명', '업종'])
        data_3 = pd.read_excel('국내펀드보유종목내역.xlsx', sheet_name='5230', usecols=['종목명', '업종'])

        corp_list = dict()

        for i in range(0, len(data_1) - 1):
            corp_list[data_1['종목명'][i]] = data_1['업종'][i]

        for i in range(0, len(data_2) - 1):
            corp_list[data_2['종목명'][i]] = data_2['업종'][i]

        for i in range(0, len(data_3) - 1):
            corp_list[data_2['종목명'][i]] = data_2['업종'][i]


    def get_reports_list(self):


        global api_key

        dart.set_api_key(api_key=api_key)

        detail_index = ["A001", "A002", "A003", "B001", "B002", "B003", "C001", "C002", "C003", "C004", "C005", "C006",
                        "C007", "C008",
                        "C009", "C010", "D001", "D002", "D004", "E001", "E003", "E004", "E008", "I001", "I002", "I003",
                        "I004", "I005", "I006"]

        detail_index_fin = ["A001", "A002", "A003", "B001", "B002", "B003",
                            "D001", "D002", "D004", "E001", "E003", "E004", "E008", "I001", "I002",
                            "I003", "I004", "I005", "I006"]

        reports_whole = dart.filings.search(page_count=100, pblntf_detail_ty=detail_index)
        for j in range(1, reports_whole.total_page + 1):
            reports = dart.filings.search(page_count=100, pblntf_detail_ty=detail_index, page_no=j)

            length_reports = len(reports.report_list)

            corp_dict = {}
            report_code = {}
            report_name = {}

        for j in range(1, reports_whole.total_page + 1):
            reports = dart.filings.search(page_count=100, pblntf_detail_ty=detail_index, page_no=j)

            length_reports = len(reports.report_list)

            for i in range(0, length_reports):
                dict_reports = reports.report_list[i].to_dict()
                if dict_reports['corp_name'] in corp_list:

                    if corp_list.get(dict_reports['corp_name']) != '증권':
                        report_code[dict_reports['rcp_no']] = dict_reports['corp_name']
                        corp_dict[dict_reports['corp_name']] = dict_reports['corp_code']
                        report_name[dict_reports['rcp_no']] = dict_reports['report_nm']

        for j in range(1, reports_whole.total_page + 1):
            reports = dart.filings.search(page_count=100, pblntf_detail_ty=detail_index_fin, page_no=j)

            length_reports = len(reports.report_list)

            fin_dict = {}

            for i in range(0, length_reports):
                dict_reports = reports.report_list[i].to_dict()

                if dict_reports['corp_name'] in corp_list:

                    if corp_list.get(dict_reports['corp_name']) == '증권':
                        report_code[dict_reports['rcp_no']] = dict_reports['corp_name']
                        fin_dict[dict_reports['corp_name']] = dict_reports['corp_code']
                        report_name[dict_reports['rcp_no']] = dict_reports['report_nm']

        return corp_dict, fin_dict




def news(corp,news_dict,chrome_path):
    # 창 숨기는 옵션 추가
    options = webdriver.ChromeOptions()
    options.add_argument("headless")
    

    driver = webdriver.Chrome(chrome_path,options=options)
    corpo = "이제부터 종목검색을 시작하겠습니다."
    URL = 'https://search.naver.com/search.naver?where=news&query=' + corpo + '&sm=tab_opt&sort=0&photo=1&field=0&pd=7&docid=&related=0&mynews=0&office_type=0&dupremove=1&office_section_code=0&news_office_checked=&nso=so%3Add%2Cp%3Aall&is_sug_officeid=0'
    driver.get(URL)
    for corp_1 in corp:
        last_news=[]
        real_news = []
        search = driver.find_element_by_xpath('// *[ @ id = "nx_query"]')
        search.clear()
        search.send_keys(corp_1)
        button = driver.find_element_by_xpath('//*[@id="nx_search_form"]/fieldset/button/i')
        button.click()
        num = len(driver.find_elements_by_css_selector(".news_tit"))

        if num !=0 and num>4:
            #print(corp_1, "뉴스개수는 %d 개 있습니다." % num)
            for i in range(1, num+1):

                news = driver.find_element_by_xpath('//*[@id="main_pack"]/section/div/div[2]/ul/li[' + str(i) + ']/div[1]/div/a')
                news = news.text
                news = news.replace("'", " ")
                news = news.replace("!", " ")
                news = news.replace(".", " ")
                news = news.replace(",", " ")

                news_list = news.split()
                # print(news_list)
                
                if corp_1 not in news_list:
                    continue
                last_news.append(news) #기업name이 들어간 뉴스기사들

            try:
                model = SentenceTransformer('paraphrase-distilroberta-base-v1') #임베딩을 하여 중복되거나 유사한 뉴스는 제거되게끔 하였습니다.
                embeddings = model.encode(last_news,convert_to_tensor=True)
                cosine_scores = util.pytorch_cos_sim(embeddings,embeddings)
                
                real_news.append(last_news[0])
                middle_news_list = []


                if corp_1 not in news_dict.keys() :

                    for index, i in enumerate(cosine_scores[0].tolist()):
                        # print(index,i)
                        # print(last_news[index])

                        if i < 0.5:
                            real_news.append(last_news[index])

                    news_dict[corp_1] = real_news
                    print(corp_1, "::", real_news)
                    #print(news_dict)

                else:
                    for newss in news_dict[corp_1]:
                        middle_news_list.append(newss)
                        model1 = SentenceTransformer('paraphrase-distilroberta-base-v1')
                        embeddings_1 = model.encode(middle_news_list, convert_to_tensor=True)
                        cosine_scores_1 = util.pytorch_cos_sim(embeddings_1, embeddings_1)
                        for index,i in enumerate(cosine_scores_1[0].tolist()):
                            if i<0.5 or i==1:
                                #print(corp_1, "::", real_news)
                                news_dict[corp_1] = middle_news_list

                time.sleep(3)


            except:
                continue




            time.sleep(2)

            #print(news_dict)

        else:

            search = driver.find_element_by_xpath('// *[ @ id = "nx_query"]')
            search.clear()
            search.send_keys(corp_1)
            button = driver.find_element_by_xpath('//*[@id="nx_search_form"]/fieldset/button/i')

            button.click()
    driver.quit()
    #print(news_dict)
    return news_dict

def call_news(news_dict):
    print(news_dict)






In [ ]:
searching_reports()